Imports


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import qrcode
from pyzbar import pyzbar
import os
from natsort import natsorted

Utilities


In [ ]:

def update_not_repeated_output(output,somme):

    prefix = "00"
    #--- convertir la taille de la chaine qui se repete pas en HEXADECIMAL
    hex_somme = str(hex(int(somme))).split('x')[1].upper()+"H"
    added_string = '('+prefix+hex_somme+')' 

    output=output.split(' ')
    #---- inseré la taille de la chaine qui se repete pas avant la premiere occurrence de la chaine
    output.insert(len(output)-1-somme,added_string)
    output = " ".join(output)
    return output


Functions

In [ ]:

def RLE_Encode(message_to_code, nb_repetition=0,bits=15):

    bit_size =  2**(bits+1)-2**bits
    separateur = "$"
    show_at_start = 0
    # ----- Ancien mot/lettre
    temp = message_to_code[0]
    # ----- Dictionnaire contenant le nb de repétitions
    dictionnary = {}
    # ----- Clé d'une dictionnaire
    lazy_index = 0
    # ----- Nombre de répétition MAX souhaiter
    repeated_letter = 0

    # ---- Pour chaque mot/lettre
    for value in message_to_code:
        # ----- Si la lettre égale a ancienne & pas répété + de Nfois
        if temp == value:   
            # ---- on verifie si le nombre de bits n'est pas depassé
            if repeated_letter < bit_size:
                repeated_letter += 1
            else :
                repeated_letter = 1
                lazy_index +=1
        # ----- Sinon changer de clé de dictionnaire
        else:
            repeated_letter = 1
            lazy_index += 1

        key = f'{lazy_index}{separateur}{value}'

        # ---- Nombre de fois que cette valeur est repeté
        dictionnary[key]=repeated_letter


        temp = value

    # ---- Contiendra le codage
    output = ""
    # ---- La taille de la liste qui se repete pas
    somme = 0
    # ---- boolean qui permet de savoir si c'est la premiere fois qu'on rencontre une liste qu'on repete pas
    first_time = True
    # ---- indice de l'emplacement du début de cette liste
    indice = -1
    # ---- si il y'a que une liste qui contient pas repetition executé le bloc quand cette variable est vrai
    not_in_else = True
    # --- boolean qui dit si on est arrivé dans une sequence qui se repete pas 
    is_nb_rep_once = False

    for key, value in dictionnary.items():
        prefix = "80"

        # ---- convertir le pixel ainsi que le nombre de repetition en hexa
        hex_value = str(hex(int(key.split(separateur)[1]))).split('x')[1].upper()+"H"
        hex_repeated = str(hex(int(value))).split('x')[1].upper()+"H"
        
        # ---- Si le nombre de répétition est inferieur a celui fixé 
        if value < nb_repetition: 
            is_nb_rep_once = True
  
            somme += value #---------- On compte le nombre de fois que le caractere est apparu
            if hex_value.startswith('0'): #----- Si la valeur de l'hexdecimal commence par 0
                for _ in range(value):
                    output+='0'+hex_value+' ' #--- on ajoute a la chaine a une variable contenant la chaine compressé ainsi qu'un 0 au début
            else:
                for _ in range(value):
                    output+=hex_value+' ' #--- Sinon on ajoute la chaine a une variable contenant la chaine compressé
        else :
            #- ----- Sinon si on est arrivé a la fin d'une sequence qui se repete pas 
            if is_nb_rep_once:
                # ---- on renitialise le boolean
                is_nb_rep_once = False
                # ---- on ajoute au début de la chaine qui se repete pas le nombre de repetition de cetter derniere
                output =  update_not_repeated_output(output,somme)
                #----- on remet le compteur a 0
                somme = 0

            if hex_value.startswith('0'):
                # ---- si la valeur HEXA commence par 0,on ajoute un 0 puis la valeur sinon direct la valeur
                output += '('+prefix+'0'+hex_repeated+') 0' + hex_value+' '
            else:
                output += '('+prefix+'0'+hex_repeated+') ' + hex_value+' '


    # ---- Si on est arrivé a la fin d'une sequence qui se repete pas on insert le nombre de repetition de cette sequence
    
    if is_nb_rep_once:
        output =  update_not_repeated_output(output,somme)

            
    return output








In [ ]:
def RLE_Decode(encoded_message,width,height,bits=15):
    
    bit_size =  2**(bits+1)-2**bits

    decoded_pixels = []

    # ---- pour chaque valeur codé et index
    for index,encoded_value in enumerate(encoded_message):
        
        # ---- si la valeur courante est le nombre de repetition
        if encoded_value.startswith('(') : 

            # --- si la valeeur courante est le nb_repetition d'une chaine qui se repete pas 
            if encoded_value.startswith('(0'):
                # ---- on convertit la valeur en decimal
                dec_value = int(encoded_value.split('(00')[1].split('H)')[0],16)

                # ---- on boucle et on ajoute les valeurs de cette sequence
                for i in range(1,dec_value+1):  
                    decoded_pixels.append(int(encoded_message[index+i].split('H')[0],16))

            elif encoded_value.startswith('(8'):
                # ---- si une sequence de repete on convertit en decimal et on ajoute la prochaine valeur codé n fois
                dec_value = int(encoded_value.split('(80')[1].split('H)')[0],16)
                next_encoded_value = int(encoded_message[index+1].split('H')[0],16)

                for _ in range(dec_value):
                    decoded_pixels.append(next_encoded_value)
        else:
            continue

    # ---- on convertit la liste en numpy array et on la reshape
    image = np.array(decoded_pixels)
    image = image.reshape(width,height)

    return image




QR Code

In [ ]:

def QR_Decode():
    big_files = []
    car_index = -5
    first_time = True
    add_value = 0

    # ----- pour les fichiers dans notr dossier 
    for files in natsorted(os.listdir()):

        last_car = files[car_index]
        # ---- si notre fichier es tun QR_CODE et pas une grande IMAGE
        if files.startswith("QR_CODE") and not last_car.isnumeric():

            # --- on lit le QRCode
            image = cv2.imread(files)
            # --- on le décode
            qr_code = pyzbar.decode(image)[0]
            data = qr_code.data.decode("utf-8")

            # --- on retire la chaien compressé
            compressed_image = data.split('$')[0].split(' ')

            # --- on récupere le height et width
            width = data.split('$')[1].split(',')[0]
            height = data.split('$')[1].split(',')[1]

            # ---- on decompresse
            decompressed_image = RLE_Decode(compressed_image,int(width),int(height))
            print("----------- QR_code-Décodé -----------\n")
            plt.imshow(decompressed_image)
            plt.show()
            print("----------- ************ -----------\n")

        elif files.startswith("QR_CODE"):
            #---- on ajoute le fichier a la liste "big_files"
           big_files.append(files)
                

    # big_files = sorted(big_files,key=lambda x : x[-5])
    temp = 0
    temp_name= big_files[0]
    compressed_image = []

    # ---- pour chaque mini QRCode de l'image (y'a bcp de QRCode car grande image)
    for index,files in enumerate(big_files):
        #--- on lit le QR
        image = cv2.imread(files)
        qr_code = pyzbar.decode(image)[0]
            
        data = qr_code.data.decode("utf-8")

        # --- on decode et extrait la chaine compressé et on la extend a l'ancienne chaine du QRCode précédent
        compressed_image.extend(data.split('$')[0].split(' '))
        compressed_image.pop()  
        # --- on recupere la shape qu'il faut pour reshape

        width = data.split('$')[1].split(',')[0]
        height = data.split('$')[1].split(',')[1]

    
        if int(files[car_index]) == 0 and not first_time :
            if add_value!=0:
                add_value += 10
            else:
                add_value = 10
       
        
            
        current_value = int(files[car_index])+add_value
        print(files,current_value,temp)
        # ---- si la sequence de QRCode qui construit l'image n'est pas fini on avance
        if current_value >= temp and index < len(big_files)-1  and big_files[index+1].split('.')[0] == files.split('.')[0]: 
            temp = int(files[car_index])+add_value
            first_time = False

        # ---- Sinon on renitialise toute les variables pour preparé la prochaine sequence de QRCode et on decode la sequence actuelle
        else:
            temp = 0
            add_value = 0
            first_time = True
            compressed_image.append("")
            decompressed_image = RLE_Decode(compressed_image,int(width),int(height))
            print("----------- QR_code-Décodé -----------\n")
            plt.imshow(decompressed_image)
            plt.show()
            print("----------- ************ -----------\n")
            compressed_image = []
        temp_name = files



Main

In [ ]:

def MainImage():

    MAX_SIZE = 2000

    liste_images = [
        "image_1.bmp",
        "image_2.bmp",
        "image_3.bmp",
        "binary_random1.png",
        "binary_cablecar1.bmp",
        # "cablecar.bmp" # en niveau de gris
    ]

    for files in liste_images:
        image_tp_3 = cv2.imread(files, 0)
        width,height = image_tp_3.shape

        print("\n----------- Image -----------\n")
        plt.imshow(image_tp_3)
        plt.show()
        print("----------- ***** -----------\n")

        # --- Applatir l'image
        string_image = image_tp_3.flatten().tolist()

        # --- convertir les éléments en strings
        string_image = [str(words) for words in string_image]

        compressed_image = RLE_Encode(string_image,3,15)

        # ---- Ecrire l'image compressée dans un fichier .txt
        with open(f"{files}.txt","w") as fichier:
            fichier.write(compressed_image)

        compressed_qr = []

        # ---- si la taille de l'image compressé est inferieur a MAX_SIZE on met la chaine dans un QRCode
        if len(compressed_image) < MAX_SIZE:
            print("Saving image into QRcode ...")
            data = compressed_image+"$"+str(width)+","+str(height)
            img = qrcode.make(data)
            img.save(f"QR_CODE_{files}.png")

        # ---- Sinon on la divise en plusieur sous chaine de taille MAX_SIZE qu'on met dans plsuieurs QRCODE 
        else:
            string_to_add = ""
            compressed_image_split = compressed_image.split(' ')

            for index,element in enumerate(compressed_image_split):
                # --- on ajoute l'element de la chaine compressé et on verifie sa taille
                string_to_add+=(element+" ")
                if len(string_to_add)<MAX_SIZE and index < len(compressed_image_split)-1:
                    continue
                else:
                    # --- si la taille depasse max_size, on ajoute la sequence pour en faire un QRCODE   et on se prepare pour une autre 
                    compressed_qr.append(string_to_add)
                    string_to_add = ""

            # ---- pour chaque sequence obtenue on crée un QRCODE qu'on numerote

            for index,qr in enumerate(compressed_qr):
                data = qr +"$"+str(width)+","+str(height)
                img = qrcode.make(data)
                img.save(f"QR_CODE_{files}_{index}.png")
        


        
        compressed_image = compressed_image.split(' ')
        compressed_image.pop()
      
        size_compressed = str("".join(compressed_image))
        #--- on retire les données inutiles a la compression
        size_compressed = size_compressed.replace("(","")
        size_compressed = size_compressed.replace(")","")
        size_compressed = size_compressed.replace("H","")

        compression_rate = (1-(len(size_compressed)/2)/len(string_image))*100

        print(f"Taux de compression : {compression_rate}")

        #---- on lit les fichiers compréssé et on les decompresses
        with open(f"{files}.txt","r") as fichier:
            compressed_image = fichier.read().split(' ')
            compressed_image.pop()

        decompressed_image = RLE_Decode(compressed_image,int(width),int(height))
        

        print("----------- Image-Décodé -----------\n")
        plt.imshow(decompressed_image)
        plt.show()
        print("----------- ************ -----------\n")

        # ---- on calcule la différence
        var = np.count_nonzero(image_tp_3-decompressed_image)
        print(f"Soustraction des deux images : \n{var}")

    QR_Decode()


In [ ]:
MainImage()
